<a href="https://colab.research.google.com/github/Rya-Sanovar/Eulerian-Video-Magnification/blob/main/EVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import cv2
import numpy as np
import scipy.signal as signal
import scipy.fftpack as fftpack

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


## Necessary Functions

In [ ]:
#convert RBG to YIQ
def rgb2yiq(src):
    [rows,cols]=src.shape[:2]
    dst=np.zeros((rows,cols,3),dtype=np.float64)
    T = np.array([[0.299, 0.587, 0.114], [0.5959, -0.2746, -0.3213], [0.2115, -0.5227, 0.3112]])
    for i in range(rows):
        for j in range(cols):
            dst[i, j]=np.dot(T,src[i,j])
    return dst

#convert YIQ to RBG
def yiq2rbg(src):
    [rows, cols] = src.shape[:2]
    dst=np.zeros((rows,cols,3),dtype=np.float64)
    T = np.array([[1, 0.956, 0.619], [1, -0.272, -0.647], [1, -1.106, 1.703]])
    for i in range(rows):
        for j in range(cols):
            dst[i, j]=np.dot(T,src[i,j])
    return dst

#Build Gaussian Pyramid
def build_gaussian_pyramid(src,level=3):
    s=src.copy()
    pyramid=[s]
    for i in range(level):
        s=cv2.pyrDown(pyramid[i])      
        pyramid.append(s)
    return pyramid

#Build Laplacian Pyramid
def build_laplacian_pyramid(src,levels=3):
    gaussianPyramid = build_gaussian_pyramid(src, levels)
    pyramid=[gaussianPyramid[-1]]
    for i in range(levels,0,-1):
        size=(gaussianPyramid[i-1].shape[1], gaussianPyramid[i-1].shape[0])
        GE=cv2.pyrUp(gaussianPyramid[i], dstsize=size)
        L=cv2.subtract(gaussianPyramid[i-1],GE)
        pyramid.append(L)
    return pyramid

#load video from file
def load_video(video_filename):
    cap=cv2.VideoCapture(video_filename)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    video_tensor=np.zeros((frame_count,height,width,3),dtype='float')
    x=0
    while cap.isOpened():
        ret,frame=cap.read()
        if ret is True:
            video_tensor[x]=frame
            x+=1
        else:
            break
    return video_tensor,fps

# apply temporal ideal bandpass filter to gaussian video
def temporal_ideal_filter(tensor,low,high,fps,axis=0):
    fft=fftpack.fft(tensor,axis=axis)
    frequencies = fftpack.fftfreq(tensor.shape[0], d=1.0 / fps)
    bound_low = (np.abs(frequencies - low)).argmin()
    bound_high = (np.abs(frequencies - high)).argmin()
    fft[:bound_low] = 0
    fft[bound_high:-bound_high] = 0
    fft[-bound_low:] = 0
    iff=fftpack.ifft(fft, axis=axis)
    return np.abs(iff)

# build gaussian pyramid for video
def gaussian_video(video_tensor,levels=3):
    for i in range(0,video_tensor.shape[0]):
        frame=video_tensor[i]
        pyr=build_gaussian_pyramid(frame,level=levels)
        gaussian_frame=pyr[-1]
        if i==0:
            vid_data=np.zeros((video_tensor.shape[0],gaussian_frame.shape[0],gaussian_frame.shape[1],3))
        vid_data[i]=gaussian_frame
    return vid_data

#amplify the video
def amplify_video(gaussian_vid,amplification=50):
    return gaussian_vid*amplification

#reconstruct video from original video and gaussian video
def reconstruct_video(amp_video,origin_video,levels=3):
    final_video=np.zeros(origin_video.shape)
    print('reconstruction')
    print(amp_video.shape)
    for i in range(0,amp_video.shape[0]):
        img = amp_video[i]
        print('img=amp_video[', i, '], img.shape = ', img.shape)
        for x in range(levels):
            # size=(origin_video[x-1].shape[1], origin_video[x-1].shape[0])
            # img=cv2.pyrUp(img, dstsize=size)
            img=cv2.pyrUp(img)
            print('after range of ', levels, '.',x,' levels, pyrUp, img.shape = ', img.shape, ', origin_video.shape = ', origin_video.shape)
        img=cv2.add(img,origin_video[i])
        final_video[i]=img
    return final_video

#save video to files
def save_video(video_tensor):
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    [height,width]=video_tensor[0].shape[0:2]
    writer = cv2.VideoWriter("out.avi", fourcc, 30, (width, height), 1)
    for i in range(0,video_tensor.shape[0]):
        writer.write(cv2.convertScaleAbs(video_tensor[i]))
    writer.release()

#magnify color
def magnify_color(video_name,low,high,levels=3,amplification=20):
    t,f=load_video(video_name)
    gau_video=gaussian_video(t,levels=levels)
    print(type(gau_video))
    print(gau_video.shape)
    filtered_tensor=temporal_ideal_filter(gau_video,low,high,f)
    print(type(filtered_tensor))
    print(filtered_tensor.shape)
    amplified_video=amplify_video(filtered_tensor,amplification=amplification)
    print(type(amplified_video))
    print(amplified_video.shape)
    final=reconstruct_video(amplified_video,t,levels=levels)
    save_video(final)

#build laplacian pyramid for video
def laplacian_video(video_tensor,levels=3):
    tensor_list=[]
    for i in range(0,video_tensor.shape[0]):
        frame=video_tensor[i]
        pyr=build_laplacian_pyramid(frame,levels=levels)
        if i==0:
            for k in range(levels):
                tensor_list.append(np.zeros((video_tensor.shape[0],pyr[k].shape[0],pyr[k].shape[1],3)))
        for n in range(levels):
            tensor_list[n][i] = pyr[n]
    return tensor_list

#butterworth bandpass filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    omega = 0.5 * fs
    low = lowcut / omega
    high = highcut / omega
    b, a = signal.butter(order, [low, high], btype='band')
    y = signal.lfilter(b, a, data, axis=0)
    return y

#reconstruct video from laplacian pyramid
def reconstruct_from_tensorlist(filter_tensor_list,levels=3):
    final=np.zeros(filter_tensor_list[-1].shape)
    for i in range(filter_tensor_list[0].shape[0]):
        up = filter_tensor_list[0][i]
        for n in range(levels-1):
            up=cv2.pyrUp(up)+filter_tensor_list[n + 1][i]
        final[i]=up
    return final

#manify motion
def magnify_motion(video_name,low,high,levels=3,amplification=20):
    t,f=load_video(video_name)
    lap_video_list=laplacian_video(t,levels=levels)
    filter_tensor_list=[]
    for i in range(levels):
        filter_tensor=butter_bandpass_filter(lap_video_list[i],low,high,f)
        filter_tensor*=amplification
        filter_tensor_list.append(filter_tensor)
    recon=reconstruct_from_tensorlist(filter_tensor_list)
    final=t+recon
    save_video(final)

## EVM

In [ ]:
#@markdown If you are using Colab, upload the video to Colab session's storage from the source
#!cp 'drive/My Drive/Self/temp/video.mp4' '.'

Make sure that the no of levels you go down the pyramid, you can go up too.<br>
<font color='red'>You can go down `N` levels till the dimension of the `N`th level is even, otherwise for odd, you may encounter an error in reconstruction.</font><br>
For eg, if image frame has a shape of `(720, 1280, 3)`, then you can go down 4 levels, where you hit odd<br>
(i.e. <br>
720/2 = 360<br>
360/2 = 180<br>
180/2 = 90<br>
90/2  = 45)

In [ ]:
#@title Using the `magnify_color` function on video

In [ ]:
%%time
# Choose the low and high for the passband. For face it is best at low=50/60 and high=60/60
magnify_color("video.mp4",low=0.83333,high=1,levels=6,amplification=50)

A file `out.avi` will be generated and saved in the Colab session storage